In [ ]:
!pip install git+https://github.com/coqui-ai/TTS

!pip install deepspeed==0.16.2
!pip install transformers==4.41.2

In [ ]:
!git lfs install
!git clone https://huggingface.co/OmarSamir/EGTTS-V0.1

In [1]:
import os
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

CONFIG_FILE_PATH = 'C:/Users/Patrickn/Jupyter_notebooks/Graduation/AI_Companion/TTS/egtts/EGTTS-V0.1/config.json'
VOCAB_FILE_PATH = 'C:/Users/Patrickn/Jupyter_notebooks/Graduation/AI_Companion/TTS/egtts/EGTTS-V0.1/vocab.json'
MODEL_PATH = 'C:/Users/Patrickn/Jupyter_notebooks/Graduation/AI_Companion/TTS/egtts/EGTTS-V0.1'
SPEAKER_AUDIO_PATH = 'C:/Users/Patrickn/Jupyter_notebooks/Graduation/AI_Companion/TTS/egtts/EGTTS-V0.1/speaker_reference.wav'

print("Loading model...")
config = XttsConfig()
config.load_json(CONFIG_FILE_PATH)
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_dir=MODEL_PATH, use_deepspeed=False, vocab_path=VOCAB_FILE_PATH)
model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

print("Computing speaker latents...")
gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path=[SPEAKER_AUDIO_PATH])

W0703 16:01:00.268000 8540 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


Loading model...


c:\ProgramData\anaconda\envs\xtts\lib\site-packages\coqpit\coqpit.py:898: UserWarning: Type mismatch in XttsConfig
Failed to deserialize field: test_sentences (typing.List[str]) = [{'text': 'وإزاي نقدر نطبق التوازن بشكل عملي على شغلنا وتفكيرنا وجسمنا وحياتنا العاطفية وأولادنا وحتى أكلنا؟', 'speaker_wav': '/content/gdrive/MyDrive/finetuning_dataset/dataset/wavs/akhdar_audio_163_chunk_186.wav', 'language': 'ar'}]
Replaced it with field's default value: []
  self.deserialize(dump_dict)


Computing speaker latents...


In [2]:
import torch
print(torch.cuda.is_available())
print(next(model.parameters()).device)

True
cuda:0


In [3]:
with open("C:/Users/Patrickn/Jupyter_notebooks/Graduation/AI_Companion/gbt/gbt_output.txt", "r", encoding="utf-8") as f:
    text = f.read().strip()
text

'بص، يا لورنزو، "واحد اتنين تلاته" ممكن تكون بداية جيدة للعد، لكن في الوضع اللي إحنا فيه دلوقتي، محتاجين نكون أكتر تركيزًا. إحنا هنا في عصر مختلف تمامًا، وفي حاجة نعملها. \n\nخلينا نفكر في الخطوة الجاية. عايزين نبدأ نراقب البيئة من حولنا، ونتعلم عن الكائنات اللي عايشة هنا، والتقاليد، وأي معلومات ممكن تفيدنا في البحث عن الـ"أنخ". \n\nهل عندك أي فكرة عن المكان اللي ممكن نبدأ منه بحثنا؟ أو أي أشخاص ممكن نتكلم معهم؟'

In [5]:
from IPython.display import Audio, display

# text = "انا اسمي باتريك و المشروع ان شاء اللة هيبقي زي الفل"
print("Inference...")

out = model.inference(
    text,
    "ar",
    gpt_cond_latent,
    speaker_embedding,
    temperature=0.75,
)

# AUDIO_OUTPUT_PATH = "/content/sample_data/output_audio.wav"
torchaudio.save("xtts_audio.wav", torch.tensor(out["wav"]).unsqueeze(0), 24000)
display(Audio(out["wav"], rate=24000, autoplay=True))

Inference...
[!] Warning: The text length exceeds the character limit of 166 for language 'ar', this might cause truncated audio.
